In [1]:
# import streamlit as st
# openai_api = st.secrets["OPENAI_API_KEY"]
openai_api = "OpenAI_API_KEY"

In [6]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex,Settings
from llama_index.core.node_parser import SentenceSplitter,SemanticSplitterNodeParser
from llama_index.llms.openai import OpenAI
from llama_index.llms.openai import OpenAI as OpenAIsum
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.storage import StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.storage.chat_store import SimpleChatStore
from llama_index.core.memory import ChatMemoryBuffer,ChatSummaryMemoryBuffer

import chromadb
import tiktoken


chat_store = SimpleChatStore()
chat_memory = ChatMemoryBuffer.from_defaults(
    token_limit=3000,
    chat_store=chat_store,
    chat_store_key="user1",
)


sum_llm = OpenAIsum(api_key=openai_api, model="gpt-3.5-turbo", max_tokens=256)
chat_summary_memory = ChatSummaryMemoryBuffer.from_defaults(
    token_limit=256,
    chat_store=chat_store,
    chat_store_key="user1",
    llm = sum_llm,
    tokenizer_fn = tiktoken.encoding_for_model("gpt-3.5-turbo").encode
)


chat_store = SimpleChatStore.from_persist_path(
    persist_path="chat_store.json"
)



documents = SimpleDirectoryReader("./data").load_data()
db = chromadb.PersistentClient(path="./vec_db")

chroma_collection = db.get_or_create_collection("quickstart")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)


In [3]:
Settings.llm = OpenAI(model="gpt-3.5-turbo",api_key=openai_api,)
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

In [7]:
vector_index = VectorStoreIndex.from_vector_store(vector_store, storage_context=storage_context,)
vector_query_engine = vector_index.as_chat_engine(chat_memory=chat_summary_memory,storage_context=storage_context,use_async=True,)

In [12]:
import os
import shutil
doc_store_path = "./doc_dir"

def filter_unsaved(file_paths:list):
    for i in file_paths:
        if os.path.isfile(os.path.join(doc_store_path,os.path.basename(i))):
            file_paths.remove(i)
            print("File already exist : {}".format(i))
        else:
            shutil.copy2(i,doc_store_path)
    return file_paths

def add_doc(file_paths:list):
    print(file_paths)
    file_paths = filter_unsaved(file_paths)
    print(file_paths)
    if len(file_paths) == 0:
        return
    docs = SimpleDirectoryReader(input_files=file_paths).load_data()
    splitter = SemanticSplitterNodeParser(buffer_size=1, breakpoint_percentile_threshold=95, embed_model=Settings.embed_model,chunk_size=256)
    nodes = splitter.get_nodes_from_documents(docs)
    vector_index2 = VectorStoreIndex(nodes)
    vector_index.insert_nodes(nodes)

In [14]:
add_doc([
    './llamaindex_doc.pdf'
])

['./llamaindex_doc.pdf']
['./llamaindex_doc.pdf']


In [15]:
q_engine = vector_query_engine

In [19]:
response = q_engine.chat("what is llamaindex?")
print(response.response)

LlamaIndex is a project that serves as a toolkit designed to easily connect LLM's with external data. It provides data structures for indexing data for various LLM tasks, data connectors to common data sources, cost transparency tools, and customizable parameters for different use cases. This allows users to query the indices in a general-purpose manner to achieve tasks typically done with an LLM, such as question-answering.


In [20]:
response = q_engine.chat("tell me how to make simple llama project inpython?")
print(response.response)

To make a simple Llama project in Python, you can follow these steps:
1. Set up a Flask backend for your project.
2. Initialize your index by writing code to load documents and create a GPTVectorStoreIndex.
3. Ensure you have the necessary dependencies like python3.11, llama_index, flask, typescript, and react.
4. Optionally, you can use a FastAPI server instead of Flask if you prefer.
5. Remember to handle user index queries in your Flask server.
6. Make sure to include the necessary code to interact with LlamaIndex and set up your server to respond to user queries effectively.
